In [5]:
import pandas as pd 

In [24]:
data = pd.read_csv('Resources/mbti_1.csv')
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [29]:
data['num_of_posts']= [len([x for x in post.split('|||') if len(x)>0]) for post in data['posts']]

In [30]:
data.head()

,type,posts,num_of_posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,50
1,ENTP,'I'm finding the lack of me in these posts ver...,50
2,INTP,'Good one _____ https://www.youtube.com/wat...,50
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",50
4,ENTJ,'You're fired.|||That's another silly misconce...,50


In [33]:
data['posts'][0].split('|||')

["'http://www.youtube.com/watch?v=qsXHcwe3krw",
 'http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg',
 'enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks',
 'What has been the most life-changing experience in your life?',
 'http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.',
 'May the PerC Experience immerse you.',
 'The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206',
 "Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...",
 '84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-ho